In [18]:
open System.Net.Http
//open System.Text.Json
//open System.Text.Json.Serialization
open System
open System.IO
open System.Net.Http.Headers
open System.Threading

In [2]:
type Credentials() =
    member val username: string = null with get, set 
    member val key: string = null with get,set
      
    static member LoadFrom(path: string): Credentials =
        use reader = new StreamReader(path)
        let json = reader.ReadToEnd()
        JsonSerializer.Deserialize<Credentials>(json)

Unhandled Exception: input.fsx (6,26)-(6,38) typecheck error The type 'StreamReader' is not defined. Maybe you want one of the following:
   Stream
input.fsx (7,20)-(7,36) typecheck error Lookup on object of indeterminate type based on information prior to this program point. A type annotation may be needed prior to this program point to constrain the type of the object. This may allow the lookup to be resolved.
input.fsx (8,9)-(8,23) typecheck error The value, namespace, type or module 'JsonSerializer' is not defined. Maybe you want one of the following:
   JsonString
   json

In [3]:
let creds = Credentials.LoadFrom("kaggle.json")
creds

Unhandled Exception: input.fsx (1,13)-(1,24) typecheck error The value, namespace, type or module 'Credentials' is not defined.

In [ ]:
let BaseApiUrl = "https://www.kaggle.com/api/v1/"

type AuthorizedClient = AuthorizedClient of HttpClient

In [ ]:
let CreateAuthorizedClient(auth: Credentials) =
    let authToken =
        sprintf "%s:%s" auth.username auth.key
        |> Text.ASCIIEncoding.ASCII.GetBytes
        |> Convert.ToBase64String

    let client = new HttpClient()
    client.DefaultRequestHeaders.Authorization <- AuthenticationHeaderValue("Basic", authToken)

    AuthorizedClient client

let DownLoadFileAsync (urlPath: string[]) destinationFolder (AuthorizedClient client) =
    let url = sprintf "%sdatasets/download/%s" BaseApiUrl <| String.Join("/", urlPath)
    let filename = urlPath |> Array.last

    async {
        use! stream = client.GetStreamAsync(url) |> Async.AwaitTask
        use fstream = new FileStream(Path.Combine(destinationFolder, filename), FileMode.CreateNew)
        let! _ = stream.CopyToAsync fstream |> Async.AwaitTask
        fstream.Close()
        stream.Close()
    }

In [ ]:
let authClient = CreateAuthorizedClient creds
let (AuthorizedClient client) = authClient

In [ ]:
DownLoadFileAsync [|"selfishgene";"historical-hourly-weather-data.zip"|] "./" authClient
|> Async.RunSynchronously


In [7]:
let DownloadFileAsync 
        (url: string) (destinationFile: string) cancellationToken 
        (report: int64 * float -> unit) (client: HttpClient) =
        async {
            let bufferLength = 8192
            use! response = client.GetAsync(url, HttpCompletionOption.ResponseHeadersRead, cancellationToken)
                            |> Async.AwaitTask
            
            response.EnsureSuccessStatusCode() |> ignore

            let total =
                if response.Content.Headers.ContentLength.HasValue
                then response.Content.Headers.ContentLength.Value
                else -1L
                |> float

            use! contentStream = response.Content.ReadAsStreamAsync() |> Async.AwaitTask
            use fileStream =
                new FileStream(destinationFile, FileMode.Create, FileAccess.Write, FileShare.None, bufferLength, true)

            let mutable totalRead = 0L
            let mutable totalReads = 0L
            let mutable isMoreToRead = true
            let buffer = Array.create bufferLength 0uy

            while isMoreToRead && not cancellationToken.IsCancellationRequested do
                let! read = contentStream.ReadAsync(buffer, 0, bufferLength) |> Async.AwaitTask
                if read > 0 then
                    do! fileStream.WriteAsync(buffer, 0, read) |> Async.AwaitTask
                    totalRead <- totalRead + int64 read
                    totalReads <- totalReads + 1L
                else
                    isMoreToRead <- false

                report (totalRead, float totalRead / total)
        }

In [9]:
let progressBar className =     
    sprintf "<div class='%s progress'>
                 <div class='progress-bar' role='progressbar'
                      style='width: 1%%;' aria-valuenow='1' aria-valuemin='0'
                      aria-valuemax='100'>1%%</div>
             </div>" className
    |> HtmlString 

let updateProgressBar className value =
    sprintf "$('.%s.progress .progress-bar')
                .css('width','%.02f%%')
                .prop('aria-valuenow',%.02f)
                .text('%.02f%%');" className value value value
    |> Javascript    

In [13]:
open System
open Microsoft.DotNet.Interactive.FSharp
open Microsoft.DotNet.Interactive
open Microsoft.AspNetCore.Html 
open FSharpKernelHelpers 

let progLog() =  
    progressBar "the-bar" 
    |> display 
    |> ignore
    
    let guid = sprintf "code-%O" <| Guid.NewGuid()
    sprintf "<div class='%s'></div>" guid
    |> HtmlString
    |> display
    |> ignore
    
    fun (bytesRead : int64,percentage: float) ->
        updateProgressBar "the-bar" (percentage*100.0)    
        sprintf "$('.%s').closest('.output').find('.output_area').each((i,el)=>{ if (i > 1) $(el).remove(); });" guid
        |> Javascript      
        

In [22]:
DownloadFileAsync 
    "https://allhdwallpapers.com/wp-content/uploads/2015/06/The-Witcher-3-game-2.png" "test.png" 
    (CancellationToken()) (progLog()) (new HttpClient())
|> Async.RunSynchronously

img.["src","test.png"] |> display

1%

In [ ]:
open FSharpPocketViewTags 

let inline (=>) (key:string) value = key,box value

type Style([<ParamArray>]props: (string*obj)[]) =
    let _props = props
    override _.ToString() =
        let result = Array.fold (fun state (key,value) -> sprintf "%s; %s: %O" state key value) null _props
        result.Trim([|';';' '|])
    member x.ToTuple() =
        "style" => x.ToString()

let prkogressBar className = 
    div.["class",className].
        ["style","border: solid orange 1px; border-radius: 5px;padding: 0"].
        innerHTML (
            span.["class","pbar"].
                 ["style","background-color:lime;text-align: right; margin: -1px;"].
                 innerHTML
                    (span.["class","label-text"].innerHTML("0.0 %") )
        )
        
div.["id","Scar"].
    ["style",Style("border"=>"solid red 1px","padding"=>"10px")].
    innerHTML
        [div.["style",Style("border"=>"solid green 3px","padding"=>"10px")]
         div.["style", "padding:10px;background-color:blue"]]
//|> display

Javascript("$('#Scar>div:last-child').css('background-color','brown').text(jQuery.fn.jquery);")